# Setup

Python version: 3.12.6

Primero se deben instalar todos los componentes para que el código funcione.

In [ ]:
%pip install groq
%pip install tenacity

# API Handling

Se configuran los parametros de uso de la API de groq

In [2]:
# API KEY para Bustio
GROQ_API_KEY = "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"

# API KEY para Vitali
#GROQ_API_KEY = "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"

# Import libraries

In [3]:
import os

import groq
import tenacity

# Create the Groq client

In [4]:
# Create a client
client = groq.Client(
    #api_key=os.environ.get("GROQ_API_KEY"), # Si se quiere usar la API Key exportada al SO.
    api_key =GROQ_API_KEY,
)

print(client)

# Using the Client created

In [12]:
# Define a function to handle chat completion
@tenacity.retry(wait=tenacity.wait_exponential(min=1, max=10), stop=tenacity.stop_after_attempt(3))
def chat_completion(prompts, model):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=prompts,
            temperature=1,
            max_tokens=4096,
            top_p=1,
            stream=True
        )
        return response
    except groq.RateLimitError:
        print("Rate limit exceeded. Please try again later.")
    except groq.AuthenticationError:
        print("Authentication failed. Please check your API key.")
    except groq.GroqError as e:
        print(f"Error: {e}")

# Using the Client created

In [14]:
#model = "llama3-8b-8192" # Erroneous model for rise exceptions
model="llama-3.1-70b-versatile"
system_prompt = "You are a weather advisor. Answer all the questions."
user_prompt = "Hello, what's the weather like today?"

messages=[
            {
                'role': 'system',
                'content': system_prompt,
            },
            {
                'role': 'user',
                'content': user_prompt,
            }
        ]

response = chat_completion(messages, model)

In [16]:
for chunk in response:
    print(chunk.choices[0].delta.content or "", end="")

However, I need a bit more information to give you an accurate forecast. Can you please tell me your location or the city you're interested in? That way, I can give you the current weather conditions, as well as any forecast updates for the day.

Also, please note that my knowledge may not be up to the minute, as my data cutoff is December 2023. But I'll do my best to provide you with a general idea of what you can expect.

# Some examples

## Extracting relevant kwywords/phrases

In [17]:
model="llama-3.1-70b-versatile"

system_prompt = "From the user text I need you to extract the relevant words."
user_prompt = "Hillary Clinton agrees with John McCain by voting to give George Bush the benefit of the doubt on Iran."

messages=[
            {
                'role': 'system',
                'content': system_prompt,
            },
            {
                'role': 'user',
                'content': user_prompt,
            }
        ]

response = chat_completion(messages, model)

In [18]:
for chunk in response:
    print(chunk.choices[0].delta.content or "", end="")

The relevant words extracted from the text are:

1. Hillary Clinton
2. John McCain
3. George Bush
4. Iran

## Extracting languages in text

In [27]:
model="llama-3.1-70b-versatile"

system_prompt = "From the given text, detect all the languages used and identify the majority language. \
    For example, for the text 'You are linda, ma chérie', the languages are: English, Spanish, French. \
        Your response should be only the majority language followed by a list of all detected languages \
            in the format: ['en', 'spa', 'fr']. Do not provide any additional explanation."


#user_prompt = "Hillary Clinton agrees with John McCain by voting to give George Bush the benefit of the doubt on Iran."
user_prompt = "Living la vida loca ma chere amie et ego sum vir molto bene!"

messages=[
            {
                'role': 'system',
                'content': system_prompt,
            },
            {
                'role': 'user',
                'content': user_prompt,
            }
        ]

response = chat_completion(messages, model)

In [28]:
for chunk in response:
    print(chunk.choices[0].delta.content or "", end="")

Spanish,['es', 'fr', 'la']